In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ops.ops import load_json, save_json
from ops.gee import export_opt_rgb, export_sar_rgb, export_cloud_map, export_opt_2a, export_sar
from ops.gee import export_edge, export_cirrus, export_opt_1c, export_cloud_scl
import os
import fiona
from tqdm.notebook import tqdm_notebook
import datetime


In [3]:
import ee
import geemap

In [4]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AdQt8qhUO7-HZXYVG2KcgqtIcj_pEGoqDvf4fOEejKJvlRPwbhmNgxYB0nY



Successfully saved authorization token.


In [5]:
paths = load_json(os.path.join('conf', 'paths.json'))
shp = load_json(os.path.join('conf', 'shp.json'))
shp_path = paths['shp']
max_cloud_cover = 95
max_thin_cirrus = 0.1
delta_days = 10
meta_path = os.path.join(paths['img'], 'metadata')
tiles_shp = os.path.join(shp_path, shp['tiles'])

In [6]:
#Create a new GEE field with the absolute distance (in days) to a given date
def prep_fn(date):
    def diff_date(image):
        img = image.set(
            'dateDist',
            ee.Number(image.get('system:time_start')).divide(60*60*24*1000).subtract(datetime.datetime.strptime(date, '%Y-%m-%d').timestamp()/(60*60*24)).abs().ceil()
        )
        img = img.set(
            'date',
            ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
        )
        img = img.set(
            'jday',
            ee.Date(image.get('system:time_start')).format('YYDDD')
        )
        return img
    return diff_date

In [7]:
def mask2clouds(image):
    qa = image.select('QA60')
    cloudBits = 1<<10
    cirrusBits = 1<<11
    
    mask = qa.bitwiseAnd(cloudBits).eq(0).And(qa.bitwiseAnd(cirrusBits).eq(0))
    
    return image.updateMask(mask)

### Inspect images dates available

In [8]:
delta_days = 20

with fiona.open(tiles_shp) as grid:
    for feat in tqdm_notebook(grid, desc = 'Tiles evaluated'):
        roi = ee.Geometry(feat['geometry'])
        
        jd18 = int(feat['properties']['d_2018'])
        jd19 = int(feat['properties']['d_2019'])
        jd20 = int(feat['properties']['d_2020'])
        
        d18_1 = datetime.datetime.strptime(f'18{jd18-delta_days}', '%y%j').strftime('%Y-%m-%d')
        d18_2 = datetime.datetime.strptime(f'18{jd18+delta_days}', '%y%j').strftime('%Y-%m-%d')
        print(f'{d18_1}   {d18_2}')
        
        d19_1 = datetime.datetime.strptime(f'19{jd19-delta_days}', '%y%j').strftime('%Y-%m-%d')
        d19_2 = datetime.datetime.strptime(f'19{jd19+delta_days}', '%y%j').strftime('%Y-%m-%d')
        print(f'{d19_1}   {d19_2}')
        
        d20_1 = datetime.datetime.strptime(f'20{jd20-delta_days}', '%y%j').strftime('%Y-%m-%d')
        d20_2 = datetime.datetime.strptime(f'20{jd20+delta_days}', '%y%j').strftime('%Y-%m-%d')
        print(f'{d20_1}   {d20_2}')
        
        
        print('==========OPTICAL DATA==============')
        opt_col_2018 = ee.ImageCollection('COPERNICUS/S2').filterDate(d18_1, d18_2).filterBounds(roi)
        opt_col_2018 = opt_col_2018.map(prep_fn(d18_1))
        
        print(opt_col_2018.aggregate_histogram('date').getInfo())
        d18jday = opt_col_2018.aggregate_histogram('jday').getInfo().keys()
        opt_dates_18 = list(map(int, list(d18jday)))
        print(opt_dates_18)
        
        opt_col_2019 = ee.ImageCollection('COPERNICUS/S2').filterDate(d19_1, d19_2).filterBounds(roi)
        opt_col_2019 = opt_col_2019.map(prep_fn(d19_1))
        print(opt_col_2019.aggregate_histogram('date').getInfo())
        d19jday = opt_col_2019.aggregate_histogram('jday').getInfo().keys()
        opt_dates_19 = list(map(int, list(d19jday)))
        print(opt_dates_19)
        
        opt_col_2020 = ee.ImageCollection('COPERNICUS/S2').filterDate(d20_1, d20_2).filterBounds(roi)
        opt_col_2020 = opt_col_2020.map(prep_fn(d20_1))
        print(opt_col_2020.aggregate_histogram('date').getInfo())
        d20jday = opt_col_2020.aggregate_histogram('jday').getInfo().keys()
        opt_dates_20 = list(map(int, list(d20jday)))
        print(opt_dates_20)
        
    
        print('==========SAR DATA==============')
        sar_col_2018 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(d18_1, d18_2).filterBounds(roi)
        sar_col_2018 = sar_col_2018.map(prep_fn(d18_1))
        print(sar_col_2018.aggregate_histogram('date').getInfo())
        d18jday = sar_col_2018.aggregate_histogram('jday').getInfo().keys()
        sar_dates_18 = list(map(int, list(d18jday)))
        print(sar_dates_18)
        
        sar_col_2019 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(d19_1, d19_2).filterBounds(roi)
        sar_col_2019 = sar_col_2019.map(prep_fn(d19_1))
        print(sar_col_2019.aggregate_histogram('date').getInfo())
        d19jday = sar_col_2019.aggregate_histogram('jday').getInfo().keys()
        sar_dates_19 = list(map(int, list(d19jday)))
        print(sar_dates_19)
        
        sar_col_2020 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(d20_1, d20_2).filterBounds(roi)
        sar_col_2020 = sar_col_2020.map(prep_fn(d20_1))
        print(sar_col_2020.aggregate_histogram('date').getInfo())
        d20jday = sar_col_2020.aggregate_histogram('jday').getInfo().keys()
        sar_dates_20 = list(map(int, list(d20jday)))
        print(sar_dates_20)


Tiles evaluated:   0%|          | 0/1 [00:00<?, ?it/s]

2018-07-06   2018-08-15
2019-07-25   2019-09-03
2020-07-11   2020-08-20
==========OPTICAL DATA==============
{'2018-07-07': 6, '2018-07-12': 6, '2018-07-17': 6, '2018-07-22': 6, '2018-07-27': 6, '2018-08-01': 6, '2018-08-06': 6, '2018-08-11': 6}
[18188, 18193, 18198, 18203, 18208, 18213, 18218, 18223]
{'2019-07-27': 6, '2019-08-01': 6, '2019-08-06': 6, '2019-08-11': 6, '2019-08-16': 6, '2019-08-21': 6, '2019-08-26': 6, '2019-08-31': 6}
[19208, 19213, 19218, 19223, 19228, 19233, 19238, 19243]
{'2020-07-11': 6, '2020-07-16': 6, '2020-07-21': 6, '2020-07-26': 6, '2020-07-31': 6, '2020-08-05': 6, '2020-08-10': 6, '2020-08-15': 6}
[20193, 20198, 20203, 20208, 20213, 20218, 20223, 20228]
==========SAR DATA==============
{'2018-07-12': 2, '2018-07-17': 2, '2018-07-24': 2, '2018-07-29': 2, '2018-08-05': 2, '2018-08-10': 2}
[18193, 18198, 18205, 18210, 18217, 18222]
{'2019-08-05': 2, '2019-08-12': 2, '2019-08-17': 2, '2019-08-24': 2, '2019-08-29': 2}
[19217, 19224, 19229, 19236, 19241]
{'2020-0

### Load images in the map

In [9]:
Map = geemap.Map(center=(-7.2, -60), zoom=8)
Map.addLayerControl()

tiles = geemap.shp_to_ee(tiles_shp)
Map.addLayer(tiles, {}, 'Tiles')

opt_vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 2048
}

sar_vizParams = {
  'bands': ['VV'],
  'min': -20,
  'max': 1
}

Map

Map(center=[-7.2, -60], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(…

#### 2018

In [10]:
#opt_dates_18 = [18188, 18193, 18198, 18203, 18208, 18213, 18218, 18223]
#sar_dates_18 = [18193, 18198, 18205, 18210, 18217, 18222]

for opt_img_date_18 in opt_dates_18:
    d =  datetime.datetime.strptime(f'{opt_img_date_18}', '%y%j').strftime('%Y-%m-%d')
    d_1 = datetime.datetime.strptime(f'{opt_img_date_18+1}', '%y%j').strftime('%Y-%m-%d')
    
    opt_col_2018 = ee.ImageCollection('COPERNICUS/S2').filterDate(d, d_1).filterBounds(roi)
    opt_img = opt_col_2018.mosaic()
    Map.addLayer(opt_img.clip(roi), opt_vizParams, f'opt_{opt_img_date_18}')
    #Map.addLayer(opt_img, opt_vizParams, f'opt_{opt_img_date_18}')
    
for sar_img_date_18 in sar_dates_18:
    d =  datetime.datetime.strptime(f'{sar_img_date_18}', '%y%j').strftime('%Y-%m-%d')
    d_1 = datetime.datetime.strptime(f'{sar_img_date_18+1}', '%y%j').strftime('%Y-%m-%d')
    
    sar_col_2018 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(d, d_1).filterBounds(roi)
    sar_img = sar_col_2018.mosaic()
    Map.addLayer(sar_img.clip(roi), sar_vizParams, f'sar_{sar_img_date_18}')
    #Map.addLayer(sar_img.clip(roi), sar_vizParams, f'sar_{sar_img_date_18}')
    

#### 2019

In [24]:
#opt_dates_19 = [19204, 19209, 19214, 19219, 19224, 19229, 19234, 19239]
#sar_dates_19 = [19209, 19221, 19233]

for opt_img_date_19 in opt_dates_19:
    d =  datetime.datetime.strptime(f'{opt_img_date_19}', '%y%j').strftime('%Y-%m-%d')
    d_1 = datetime.datetime.strptime(f'{opt_img_date_19+1}', '%y%j').strftime('%Y-%m-%d')
    
    opt_col_2019 = ee.ImageCollection('COPERNICUS/S2').filterDate(d, d_1).filterBounds(roi)
    opt_img = opt_col_2019.mosaic()
    Map.addLayer(opt_img.clip(roi), opt_vizParams, f'opt_{opt_img_date_19}')
    #Map.addLayer(opt_img, opt_vizParams, f'opt_{opt_img_date_19}')
    
for sar_img_date_19 in sar_dates_19:
    d =  datetime.datetime.strptime(f'{sar_img_date_19}', '%y%j').strftime('%Y-%m-%d')
    d_1 = datetime.datetime.strptime(f'{sar_img_date_19+1}', '%y%j').strftime('%Y-%m-%d')
    
    sar_col_2019 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(d, d_1).filterBounds(roi)
    sar_img = sar_col_2019.mosaic()
    Map.addLayer(sar_img.clip(roi), sar_vizParams, f'sar_{sar_img_date_19}')
    #Map.addLayer(sar_img.clip(roi), sar_vizParams, f'sar_{sar_img_date_19}')
    

#### 2020

In [14]:
#opt_dates_20 = [20209, 20214, 20219, 20224, 20229, 20234, 20239, 20244]
#sar_dates_20 = [20216, 20228, 20240]

for opt_img_date_20 in opt_dates_20:
    d =  datetime.datetime.strptime(f'{opt_img_date_20}', '%y%j').strftime('%Y-%m-%d')
    d_1 = datetime.datetime.strptime(f'{opt_img_date_20+1}', '%y%j').strftime('%Y-%m-%d')
    
    opt_col_2020 = ee.ImageCollection('COPERNICUS/S2').filterDate(d, d_1).filterBounds(roi)
    opt_img = opt_col_2020.mosaic()
    Map.addLayer(opt_img.clip(roi), opt_vizParams, f'opt_{opt_img_date_20}')
    #Map.addLayer(opt_img, opt_vizParams, f'opt_{opt_img_date_20}')
    
for sar_img_date_20 in sar_dates_20:
    d =  datetime.datetime.strptime(f'{sar_img_date_20}', '%y%j').strftime('%Y-%m-%d')
    d_1 = datetime.datetime.strptime(f'{sar_img_date_20+1}', '%y%j').strftime('%Y-%m-%d')
    
    sar_col_2020 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(d, d_1).filterBounds(roi)
    sar_img = sar_col_2020.mosaic()
    Map.addLayer(sar_img.clip(roi), sar_vizParams, f'sar_{sar_img_date_20}')
    #Map.addLayer(sar_img.clip(roi), sar_vizParams, f'sar_{sar_img_date_20}')
    

### Downloading individual images 

In [15]:
opt_dates = [18208, 18218, 18193, 19228, 19233, 19208, 20218, 20213, 20223]
sar_dates = [18210, 18222, 18198, 19229, 19241, 19217, 20218, 20212, 20224]

t_opt = []
for opt_img_date in opt_dates:
    d =  datetime.datetime.strptime(f'{opt_img_date}', '%y%j').strftime('%Y-%m-%d')
    d_1 = datetime.datetime.strptime(f'{opt_img_date+1}', '%y%j').strftime('%Y-%m-%d')
    
    opt_col = ee.ImageCollection('COPERNICUS/S2').filterDate(d, d_1).filterBounds(roi)
    opt_img = opt_col.mosaic()
    t_opt.append(export_opt_1c(opt_img, roi, f'{opt_img_date}'))
    #t_opt.append(export_opt_rgb(opt_img, roi, f'opt_{opt_img_date}'))
    
    
    
t_sar = []
for sar_img_date in sar_dates:
    d =  datetime.datetime.strptime(f'{sar_img_date}', '%y%j').strftime('%Y-%m-%d')
    d_1 = datetime.datetime.strptime(f'{sar_img_date+1}', '%y%j').strftime('%Y-%m-%d')
    
    sar_col = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(d, d_1).filterBounds(roi)
    sar_img = sar_col.mosaic()
    t_sar.append(export_sar(sar_img, roi, f'{sar_img_date}'))
    #t_sar.append(export_sar_rgb(sar_img, roi, f'sar_{sar_img_date}'))
    

### Download free-cloud mosaic

In [8]:
Map = geemap.Map(center=(-7.2, -60), zoom=10)
Map.addLayerControl()

tiles = geemap.shp_to_ee(tiles_shp)
Map.addLayer(tiles, {}, 'Tiles')

opt_vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 2048
}

sar_vizParams = {
  'bands': ['VV'],
  'min': -20,
  'max': 1
}

Map

Map(center=[-7.2, -60], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(…

In [16]:
delta_days = 20
max_cloud_cover = 5
max_thin_cirrus = 0.1

with fiona.open(tiles_shp) as grid:
    for feat in tqdm_notebook(grid, desc = 'Tiles evaluated'):
        roi = ee.Geometry(feat['geometry'])
        
        jd18 = int(feat['properties']['d_2018'])
        jd19 = int(feat['properties']['d_2019'])
        jd20 = int(feat['properties']['d_2020'])
        
        d18_1 = datetime.datetime.strptime(f'18{jd18-delta_days}', '%y%j').strftime('%Y-%m-%d')
        d18_2 = datetime.datetime.strptime(f'18{jd18+delta_days}', '%y%j').strftime('%Y-%m-%d')
        print(f'{d18_1}   {d18_2}')
        
        d19_1 = datetime.datetime.strptime(f'19{jd19-delta_days}', '%y%j').strftime('%Y-%m-%d')
        d19_2 = datetime.datetime.strptime(f'19{jd19+delta_days}', '%y%j').strftime('%Y-%m-%d')
        print(f'{d19_1}   {d19_2}')
        
        d20_1 = datetime.datetime.strptime(f'20{jd20-delta_days}', '%y%j').strftime('%Y-%m-%d')
        d20_2 = datetime.datetime.strptime(f'20{jd20+delta_days}', '%y%j').strftime('%Y-%m-%d')
        print(f'{d20_1}   {d20_2}')
        
        
        opt_col_2018 = ee.ImageCollection('COPERNICUS/S2').filterDate(d18_1, d18_2).filterBounds(roi)
        opt_col_2018 = opt_col_2018.map(prep_fn(d18_1)).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',max_cloud_cover))
        opt_2018 = opt_col_2018.map(mask2clouds).median().toUint16()
        Map.addLayer(opt_2018.clip(roi), opt_vizParams, f'Sentinel 2 Mosaic from 2018')

        export_opt_1c(opt_2018, roi, "cloud_free_2018")

        opt_col_2019 = ee.ImageCollection('COPERNICUS/S2').filterDate(d19_1, d19_2).filterBounds(roi)
        opt_col_2019 = opt_col_2019.map(prep_fn(d19_1)).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',max_cloud_cover))
        opt_2019 = opt_col_2019.map(mask2clouds).median().toUint16()
        Map.addLayer(opt_2019.clip(roi), opt_vizParams, f'Sentinel 2 Mosaic from 2019')

        export_opt_1c(opt_2019, roi, 'cloud_free_2019')

        opt_col_2020 = ee.ImageCollection('COPERNICUS/S2').filterDate(d20_1, d20_2).filterBounds(roi)
        opt_col_2020 = opt_col_2020.map(prep_fn(d20_1)).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',max_cloud_cover))
        opt_2020 = opt_col_2020.map(mask2clouds).median().toUint16()
        Map.addLayer(opt_2020.clip(roi), opt_vizParams, f'Sentinel 2 Mosaic from 2020')

        export_opt_1c(opt_2020, roi, 'cloud_free_2020')
        
        

Tiles evaluated:   0%|          | 0/1 [00:00<?, ?it/s]

2018-07-06   2018-08-15
2019-07-25   2019-09-03
2020-07-11   2020-08-20
